In [1]:
import pandas as pd
d = pd.read_csv("Youtube-spam-collection-v1/Youtube01-Psy.csv")

C:\Users\Yashwardhan Tulsyan\AppData\Local\Temp\ipykernel_21416\335248237.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
d.tail()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
345,z13th1q4yzihf1bll23qxzpjeujterydj,Carmen Racasanu,2014-11-14T13:27:52,How can this have 2 billion views when there's...,0
346,z13fcn1wfpb5e51xe04chdxakpzgchyaxzo0k,diego mogrovejo,2014-11-14T13:28:08,I don't now why I'm watching this in 2014﻿,0
347,z130zd5b3titudkoe04ccbeohojxuzppvbg,BlueYetiPlayz -Call Of Duty and More,2015-05-23T13:04:32,subscribe to me for call of duty vids and give...,1
348,z12he50arvrkivl5u04cctawgxzkjfsjcc4,Photo Editor,2015-06-05T14:14:48,hi guys please my android photo editor downloa...,1
349,z13vhvu54u3ewpp5h04ccb4zuoardrmjlyk0k,Ray Benich,2015-06-05T18:05:16,The first billion viewed this because they tho...,0


In [4]:
len(d.query('CLASS == 1'))

175

In [5]:
len(d.query('CLASS == 0'))

175

In [6]:
len(d)

350

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [8]:
dvec = vectorizer.fit_transform(d['CONTENT'])

In [9]:
dvec

<350x1418 sparse matrix of type '<class 'numpy.int64'>'
	with 4354 stored elements in Compressed Sparse Row format>

In [10]:
analyse = vectorizer.build_analyzer()

In [11]:
print(d['CONTENT'][349])
analyse(d['CONTENT'][349])

The first billion viewed this because they thought it was really cool, the  other billion and a half came to see how stupid the first billion were...﻿


['the',
 'first',
 'billion',
 'viewed',
 'this',
 'because',
 'they',
 'thought',
 'it',
 'was',
 'really',
 'cool',
 'the',
 'other',
 'billion',
 'and',
 'half',
 'came',
 'to',
 'see',
 'how',
 'stupid',
 'the',
 'first',
 'billion',
 'were']

In [14]:
vectorizer.get_feature_names_out()

array(['00', '000', '02', ..., 'ｆａｎｃy', 'ｉｓ', 'ｔｈｉｓ'], dtype=object)

In [15]:
dshuf = d.sample(frac=1)

In [22]:
d_train = dshuf[:300]
d_test = dshuf[300:]
d_train_att = vectorizer.fit_transform(d_train['CONTENT'])
d_test_att = vectorizer.transform(d_test['CONTENT'])
d_train_label = d_train['CLASS']
d_test_label = d_test['CLASS']

In [23]:
d_train_att

<300x1342 sparse matrix of type '<class 'numpy.int64'>'
	with 3826 stored elements in Compressed Sparse Row format>

In [24]:
d_test_att

<50x1342 sparse matrix of type '<class 'numpy.int64'>'
	with 448 stored elements in Compressed Sparse Row format>

In [25]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=80)

In [26]:
clf.fit(d_train_att, d_train_label)

RandomForestClassifier(n_estimators=80)

In [27]:
clf.score(d_test_att, d_test_label)

0.98

In [29]:
from sklearn.metrics import confusion_matrix
pred_labels = clf.predict(d_test_att)
confusion_matrix(d_test_label, pred_labels)

array([[25,  0],
       [ 1, 24]], dtype=int64)

In [30]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, d_train_att, d_train_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.95 (+/- 0.02)


In [31]:
d = pd.concat([pd.read_csv("Youtube-spam-collection-v1/Youtube01-Psy.csv"),
               pd.read_csv("Youtube-spam-collection-v1/Youtube02-KatyPerry.csv"),
               pd.read_csv("Youtube-spam-collection-v1/Youtube03-LMFAO.csv"),
               pd.read_csv("Youtube-spam-collection-v1/Youtube04-Eminem.csv"),
               pd.read_csv("Youtube-spam-collection-v1/Youtube05-Shakira.csv")])

In [32]:
len(d)

1956

In [33]:
len(d.query('CLASS == 1'))

1005

In [34]:
len(d.query('CLASS == 0'))

951

In [35]:
dshuf = d.sample(frac=1)
d_content = dshuf['CONTENT']
d_label = dshuf['CLASS']

In [36]:
from sklearn.pipeline import Pipeline, make_pipeline
pipeline = Pipeline([
    ('bag-of-words', CountVectorizer()),
    ('random forest', RandomForestClassifier()),
])
pipeline

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])

In [37]:
make_pipeline(CountVectorizer(), RandomForestClassifier())

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('randomforestclassifier', RandomForestClassifier())])

In [38]:
pipeline.fit(d_content[:1500], d_label[:1500])

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])

In [39]:
pipeline.score(d_content[1500:], d_label[1500:])

0.9539473684210527

In [40]:
pipeline.predict(["what a neat video!"])

array([0], dtype=int64)

In [41]:
pipeline.predict(["plz subscribe to my channel"])

array([1], dtype=int64)

In [42]:
scores = cross_val_score(pipeline, d_content, d_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.96 (+/- 0.01)


In [43]:
from sklearn.feature_extraction.text import TfidfTransformer
pipeline2 = make_pipeline(CountVectorizer(),
                          TfidfTransformer(norm=None),
                          RandomForestClassifier())

In [44]:
scores = cross_val_score(pipeline2, d_content, d_label, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() *2))

Accuracy: 0.96 (+/- 0.01)


In [45]:
pipeline2.steps

[('countvectorizer', CountVectorizer()),
 ('tfidftransformer', TfidfTransformer(norm=None)),
 ('randomforestclassifier', RandomForestClassifier())]

In [46]:
parameters = {
    'countvectorizer__max_features' : (None, 1000, 2000),
    'countvectorizer__ngram_range' : ((1, 1), (1, 2)),
    'countvectorizer__stop_words' : ('english', None),
    'tfidftransformer__use_idf' : (True, False),
    'randomforestclassifier__n_estimators' : (20, 50, 100)
}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipeline2, parameters, n_jobs=1, verbose=1)

In [47]:
grid_search.fit(d_content, d_label)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(estimator=Pipeline(steps=[('countvectorizer', CountVectorizer()),
                                       ('tfidftransformer',
                                        TfidfTransformer(norm=None)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier())]),
             n_jobs=1,
             param_grid={'countvectorizer__max_features': (None, 1000, 2000),
                         'countvectorizer__ngram_range': ((1, 1), (1, 2)),
                         'countvectorizer__stop_words': ('english', None),
                         'randomforestclassifier__n_estimators': (20, 50, 100),
                         'tfidftransformer__use_idf': (True, False)},
             verbose=1)

In [48]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for params_name in sorted(parameters.keys()):
    print("\t%s: %r" % (params_name, best_parameters[params_name]))

Best score: 0.965
Best parameters set:
	countvectorizer__max_features: 2000
	countvectorizer__ngram_range: (1, 2)
	countvectorizer__stop_words: 'english'
	randomforestclassifier__n_estimators: 50
	tfidftransformer__use_idf: True
